In [103]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Cargar y preparar los datos
def load_and_prepare_data(file_path):
    df = pd.read_csv(file_path, index_col=0)  # La primera columna es el índice
    
    # Normalizar los datos
    scaler = MinMaxScaler(feature_range=(0, 1))
    sales_data_scaled = scaler.fit_transform(df)
    
    return sales_data_scaled, scaler, df.index

# Crear secuencias de datos para el LSTM
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(data.shape[0]):  # Iterar sobre cada producto
        for j in range(data.shape[1] - n_steps):
            X.append(data[i, j:j+n_steps])
            y.append(data[i, j+n_steps])
    return np.array(X), np.array(y)

# Construir el modelo LSTM
def build_model(n_steps):
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(n_steps, 1), return_sequences=True),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Función principal
def main():
    # Cargar y preparar los datos
    data, scaler, product_indices = load_and_prepare_data('cisco_forecast_data.csv')
    
    # Parámetros
    n_steps = 4  # Usaremos 4 trimestres para predecir el siguiente
    
    # Crear secuencias
    X, y = create_sequences(data, n_steps)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    # Dividir en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Construir y entrenar el modelo
    model = build_model(n_steps)
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)
    
    # Hacer predicciones para el próximo trimestre
    predictions = []
    for i in range(data.shape[0]):
        last_sequence = data[i, -n_steps:].reshape(1, n_steps, 1)
        predicted_scaled = model.predict(last_sequence)
        predictions.append(predicted_scaled[0, 0])
    
    # Desnormalizar las predicciones
    predictions_array = np.array(predictions).reshape(-1, 1)
    dummy_array = np.zeros((predictions_array.shape[0], data.shape[1]))
    dummy_array[:, -1] = predictions_array.flatten()
    predicted_sales = scaler.inverse_transform(dummy_array)[:, -1]
    
    # Imprimir las predicciones
    for i, sales in enumerate(predicted_sales):
        print(f"Producto {product_indices[i]}: Ventas previstas para el próximo trimestre: {sales:.2f}")

if __name__ == "__main__":
    main()

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0924 - val_loss: 0.0257
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0786 - val_loss: 0.0231
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1006 - val_loss: 0.0210
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0712 - val_loss: 0.0194
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0803 - val_loss: 0.0180
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0869 - val_loss: 0.0168
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0502 - val_loss: 0.0158
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0491 - val_loss: 0.0149
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0438 - val_loss: 0.0139
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0309 - val_loss: 0.0130
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0307 - val_loss: 0.0117
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0192 - val_loss: 0.0094


In [111]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')

# Cargar y preparar los datos
def load_and_prepare_data(file_path):
    df = pd.read_csv(file_path, index_col=0)
    scaler = MinMaxScaler(feature_range=(0, 1))
    sales_data_scaled = scaler.fit_transform(df)
    return sales_data_scaled, scaler, df.index

# Crear secuencias de datos para el LSTM
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(data.shape[0]):
        for j in range(data.shape[1] - n_steps):
            X.append(data[i, j:j+n_steps])
            y.append(data[i, j+n_steps])
    return np.array(X), np.array(y)

# Función para crear el modelo LSTM
def create_model(lstm_units=50, learning_rate=0.001):
    model = Sequential([
        LSTM(lstm_units, activation='relu', input_shape=(n_steps, 1), return_sequences=True),
        LSTM(lstm_units, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# Función principal
def main():
    # Cargar y preparar los datos
    data, scaler, product_indices = load_and_prepare_data('cisco_forecast_data.csv')
    
    global n_steps
    n_steps = 4  # Usaremos 4 trimestres para predecir el siguiente
    
    # Crear secuencias
    X, y = create_sequences(data, n_steps)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    # Definir el modelo para GridSearchCV
    model = KerasRegressor(model=create_model, verbose=0)
    
    # Definir los hiperparámetros a buscar
    param_grid = {
        'model__lstm_units': [30, 50, 70],
        'model__learning_rate': [0.001, 0.01, 0.1],
        'batch_size': [16, 32, 64],
        'epochs': [50, 100, 150]
    }
    
    # Configurar GridSearchCV
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=TimeSeriesSplit(n_splits=3), n_jobs=-1, verbose=2)
    
    # Ejecutar Grid Search
    grid_result = grid.fit(X, y)
    
    # Imprimir los mejores parámetros
    print("Mejores parámetros encontrados: ", grid_result.best_params_)
    print("Mejor puntuación: ", grid_result.best_score_)
    
    # Usar el mejor modelo para hacer predicciones
    best_model = grid_result.best_estimator_.model_
    
    # Hacer predicciones para el próximo trimestre
    predictions = []
    for i in range(data.shape[0]):
        last_sequence = data[i, -n_steps:].reshape(1, n_steps, 1)
        predicted_scaled = best_model.predict(last_sequence)
        predictions.append(predicted_scaled[0, 0])
    
    # Desnormalizar las predicciones
    predictions_array = np.array(predictions).reshape(-1, 1)
    dummy_array = np.zeros((predictions_array.shape[0], data.shape[1]))
    dummy_array[:, -1] = predictions_array.flatten()
    predicted_sales = scaler.inverse_transform(dummy_array)[:, -1]
    
    # Imprimir las predicciones
    for i, sales in enumerate(predicted_sales):
        print(f"Producto {product_indices[i]}: Ventas previstas para el próximo trimestre: {sales:.2f}")

if __name__ == "__main__":
    main()

Fitting 3 folds for each of 81 candidates, totalling 243 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Py

[CV] END batch_size=32, epochs=100, model__learning_rate=0.1, model__lstm_units=70; total time=   4.4s
[CV] END batch_size=32, epochs=150, model__learning_rate=0.01, model__lstm_units=70; total time=   6.1s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.001, model__lstm_units=50; total time=   2.3s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.001, model__lstm_units=70; total time=   2.8s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.01, model__lstm_units=70; total time=   2.4s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.1, model__lstm_units=50; total time=   2.7s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.001, model__lstm_units=50; total time=   3.9s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.01, model__lstm_units=50; total time=   3.7s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.1, model__lstm_units=70; total time=   4.2s
[CV] END batch_size=64, epochs=150, model__learning_rate=0.01, model

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Py

[CV] END batch_size=64, epochs=50, model__learning_rate=0.001, model__lstm_units=70; total time=   2.3s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.1, model__lstm_units=50; total time=   2.4s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.1, model__lstm_units=70; total time=   2.9s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.001, model__lstm_units=70; total time=   4.0s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.1, model__lstm_units=30; total time=   3.4s
[CV] END batch_size=64, epochs=150, model__learning_rate=0.001, model__lstm_units=30; total time=   4.4s
[CV] END batch_size=64, epochs=150, model__learning_rate=0.01, model__lstm_units=50; total time=   4.7s
[CV] END batch_size=64, epochs=150, model__learning_rate=0.1, model__lstm_units=50; total time=   3.6s
[CV] END batch_size=16, epochs=50, model__learning_rate=0.001, model__lstm_units=30; total time=   3.4s
[CV] END batch_size=16, epochs=50, model__learning_rate=0.1, model__

...

[CV] END batch_size=32, epochs=50, model__learning_rate=0.1, model__lstm_units=50; total time=   2.8s
[CV] END batch_size=32, epochs=100, model__learning_rate=0.01, model__lstm_units=70; total time=   4.0s
[CV] END batch_size=32, epochs=100, model__learning_rate=0.1, model__lstm_units=70; total time=   3.9s
[CV] END batch_size=32, epochs=150, model__learning_rate=0.01, model__lstm_units=30; total time=   4.4s
[CV] END batch_size=32, epochs=150, model__learning_rate=0.01, model__lstm_units=70; total time=   6.2s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.001, model__lstm_units=30; total time=   2.3s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.001, model__lstm_units=70; total time=   3.0s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.1, model__lstm_units=30; total time=   2.7s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.001, model__lstm_units=30; total time=   3.5s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.01, model

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Py

[CV] END batch_size=32, epochs=100, model__learning_rate=0.001, model__lstm_units=70; total time=   4.6s
[CV] END batch_size=32, epochs=150, model__learning_rate=0.001, model__lstm_units=50; total time=   4.4s
[CV] END batch_size=32, epochs=150, model__learning_rate=0.01, model__lstm_units=50; total time=   4.5s
[CV] END batch_size=32, epochs=150, model__learning_rate=0.1, model__lstm_units=50; total time=   5.0s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.001, model__lstm_units=50; total time=   2.4s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.01, model__lstm_units=50; total time=   2.4s
[CV] END batch_size=64, epochs=50, model__learning_rate=0.1, model__lstm_units=50; total time=   2.7s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.001, model__lstm_units=50; total time=   3.7s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.01, model__lstm_units=50; total time=   3.7s
[CV] END batch_size=64, epochs=100, model__learning_rate=0.1, mod

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Mejores parámetros encontrados:  {'batch_size': 64, 'epochs': 100, 'model__learning_rate': 0.1, 'model__lstm_units': 50}
Mejor puntuación:  0.8244544791258139
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Producto 0: Ventas previstas para el próximo trimestre: 117612.85
Producto 1: Ventas previstas para